In [1]:
#coding=utf-8
from __future__ import print_function
from numpy import nan as NA
import os,sys
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
from sklearn import cross_validation
import matplotlib.pylab as plt
from sklearn.preprocessing import Imputer
from sklearn import cross_validation, metrics
import operator
import time
from sklearn.svm import SVC
from hyperopt import hp,fmin,rand,tpe,space_eval
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.preprocessing import OneHotEncoder
from sklearn.externals import joblib
from functools import partial
from collections import Counter
from imblearn.ensemble import EasyEnsemble
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn import preprocessing

/root/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/root/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
data = pd.read_excel('../data.xlsx')

datas = data.copy()
datas=datas.sample(frac=1)

test_data =  pd.read_excel('../test_1.xlsx')


In [3]:
for i in datas.columns[11:55]:
    if datas[i].std()<=0.05:
        datas=datas.drop([i],axis=1)
        test_data=test_data.drop([i],axis=1)

In [5]:
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2,mutual_info_classif
Y = datas['Cover_Type']
X = datas.drop(['Cover_Type','Ground_position'],axis=1)
#a = SelectKBest(mutual_info_classif).fit_transform(X, Y)

In [6]:
X.shape

(100000, 46)

In [7]:
train_x_new,val_x_new,train_y_new,val_y_new = train_test_split(X,Y,test_size = 0.25,random_state=1)
def xgb_train_0901_new_feature_select(args):  
    global train_x_new,val_x_new,train_y_new,val_y_new,count,epoch_acc
    params = args
    params['max_depth'] = params.get('max_depth')+17
    params['n_estimators'] = params.get('n_estimators') * 4 + 50
    params['reg_lambda'] = max(1,params.get('reg_lambda'))
    params['subsample'] = params.get('subsample') * 0.1 + 0.7
    params['seed'] = max(1,params.get('seed') )* 100
    if count >0:
        print('\n')
    print(params)
    
    params.setdefault('objective','multi:softmax')
    params.setdefault('num_class',7)
    params.setdefault('silent',1)
    params.setdefault('eval_metric','merror')
#     params.setdefault('seed',900)
    params.setdefault('max_delta_step',30)
    

    try:
        clf = XGBClassifier(**params)
        model_bst =  clf.fit(train_x_new, train_y_new)
#        model_bst = xgb.train(params, d_train_0820, 800)
#        ypred = model_bst.predict(val_x_0821)
#        ypred = model_bst.predict(d_test_0820, ntree_limit=model_bst.best_ntree_limit)
    except:
        print('error!')
        score=0
    else:
        ypred = model_bst.predict(val_x_new)
        count = count + 1
        score = metrics.f1_score(val_y_new,ypred,average='weighted')#(val_y.values, y_test_by)
        if epoch_acc < score:
            joblib.dump(model_bst,'xgb_train_sklearn_0905.model')
            epoch_acc = score
        print("the %s times, the accuary of test dataset：" % str(count),score)
        print("Time consuming: ",round(time.time()-start_time,2),"(s)")
    return -score
      

space = {"max_depth":hp.randint("max_depth",9),
         "n_estimators":hp.randint("n_estimators",10),  #[0,1,2,3,4,5] -> [50,]
         "learning_rate":hp.uniform("eta",0.01, 0.15),  
         "subsample":hp.randint("subsample",4),#[0,1,2,3] -> [0.7,0.8,0.9,1.0]
         "reg_lambda":hp.randint("reg_lambda",15), #
         "reg_alpha":hp.randint("reg_alpha",3), 
         "scale_pos_weight":hp.uniform("scale_pos_weight", 0.1, 1),
         "gamma":hp.uniform("gamma", 0.1,0.9),
         "colsample_bytree":hp.uniform("colsample_bytree", 0.1,1),
         "seed":hp.randint("seed",12),
        }
start_time = time.time()
count = 0
epoch_acc = 0
print('=============================begin=============================')
best = fmin(xgb_train_0901_new_feature_select, space ,algo = partial(tpe.suggest,n_startup_jobs=1),max_evals=20)
space_eval(space,best)
print('\n\n'+"Bayesian hyperopt acc :    %f" % - xgb_train_0901_new_feature_select(space_eval(space , best)))
cost_time = time.time()-start_time
print ('\n'+"Bayesian hyperopt on xgboost finished!",'\n',"the total elapsed time :",round(cost_time,2),"(s)")
print('the best params: ','\n',best)
print('=============================over=============================')
'''
{'colsample_bytree': 0.9959746599126897, 'gamma': 0.16765591708467997, 'learning_rate': 0.14290452895531167, 'max_depth': 23, 'n_estimators': 70, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 0.6979891053937575, 'seed': 500, 'subsample': 1.0}
the 21 times, the accuary of test dataset： 0.7008286469373489
'''

=============================begin=============================
{'colsample_bytree': 0.10287654667498204, 'gamma': 0.5924489748707219, 'learning_rate': 0.03435745131019038, 'max_depth': 18, 'n_estimators': 66, 'reg_alpha': 2, 'reg_lambda': 13, 'scale_pos_weight': 0.5369208778723543, 'seed': 600, 'subsample': 0.8999999999999999}


/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


the 1 times, the accuary of test dataset： 0.4078306361766457
Time consuming:  12.52 (s)


{'colsample_bytree': 0.11794394414412407, 'gamma': 0.6087550041181733, 'learning_rate': 0.017508530576251957, 'max_depth': 18, 'n_estimators': 66, 'reg_alpha': 2, 'reg_lambda': 13, 'scale_pos_weight': 0.5444523071366162, 'seed': 600, 'subsample': 0.8999999999999999}


/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/root/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


the 2 times, the accuary of test dataset： 0.40789318394285207
Time consuming:  27.2 (s)


{'colsample_bytree': 0.7062182773608506, 'gamma': 0.8775677423587768, 'learning_rate': 0.015372195107675577, 'max_depth': 24, 'n_estimators': 66, 'reg_alpha': 1, 'reg_lambda': 12, 'scale_pos_weight': 0.9554379329818888, 'seed': 100, 'subsample': 0.8999999999999999}


...

the 19 times, the accuary of test dataset： 0.7026045241249734
Time consuming:  2707.53 (s)


{'colsample_bytree': 0.7788929567743115, 'gamma': 0.6761585426102682, 'learning_rate': 0.09865924640892751, 'max_depth': 21, 'n_estimators': 82, 'reg_alpha': 2, 'reg_lambda': 5, 'scale_pos_weight': 0.6426984687745715, 'seed': 900, 'subsample': 0.7999999999999999}


/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


the 20 times, the accuary of test dataset： 0.6984851672080695
Time consuming:  2925.13 (s)


{'colsample_bytree': 0.6969039980162184, 'gamma': 0.7634857002597908, 'learning_rate': 0.05657392509824035, 'max_depth': 21, 'n_estimators': 82, 'reg_alpha': 0, 'reg_lambda': 2, 'scale_pos_weight': 0.3387068004646522, 'seed': 500, 'subsample': 0.7999999999999999}
the 21 times, the accuary of test dataset： 0.7051554293436725
Time consuming:  3140.86 (s)


Bayesian hyperopt acc :    0.705155

Bayesian hyperopt on xgboost finished! 
 the total elapsed time : 3140.87 (s)
the best params:  
 {'colsample_bytree': 0.6969039980162184, 'eta': 0.05657392509824035, 'gamma': 0.7634857002597908, 'max_depth': 4, 'n_estimators': 8, 'reg_alpha': 0, 'reg_lambda': 2, 'scale_pos_weight': 0.3387068004646522, 'seed': 5, 'subsample': 1}
=============================over=============================


/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


"\n{'colsample_bytree': 0.9959746599126897, 'gamma': 0.16765591708467997, 'learning_rate': 0.14290452895531167, 'max_depth': 23, 'n_estimators': 70, 'reg_alpha': 1, 'reg_lambda': 1, 'scale_pos_weight': 0.6979891053937575, 'seed': 500, 'subsample': 1.0}\nthe 21 times, the accuary of test dataset： 0.7008286469373489\n"

In [8]:
xgb_base= joblib.load('xgb_train_sklearn_0905.model')
sorted_idx = np.argsort(xgb_base.feature_importances_)[::-1]
for index in sorted_idx:
    print([X.columns[index],xgb_base.feature_importances_[index]])

['Horizontal_Distance_To_Fire_Points', 0.16184215]
['Hillshade_3pm', 0.10205282]
['Horizontal_Distance_To_Roadways', 0.09625572]
['Hillshade_Noon', 0.09263061]
['Hillshade_9am', 0.08588783]
['Aspect', 0.08560335]
['Elevation', 0.08523431]
['Vertical_Distance_To_Hydrology', 0.078718334]
['Horizontal_Distance_To_Hydrology', 0.07350556]
['Slope', 0.063160725]
['Wilderness_Area_1', 0.005670242]
['Wilderness_Area_3', 0.005597201]
['Soil_Type_33', 0.0042363433]
['Soil_Type_32', 0.0035290048]
['Soil_Type_29', 0.003417522]
['Wilderness_Area_2', 0.0033444816]
['Soil_Type_23', 0.003286818]
['Soil_Type_24', 0.0031061391]
['Soil_Type_31', 0.003067697]
['Soil_Type_10', 0.002817822]
['Soil_Type_2', 0.0023449813]
['Soil_Type_4', 0.0022796295]
['Soil_Type_17', 0.0022642524]
['Soil_Type_30', 0.0022181217]
['Soil_Type_13', 0.0021873678]
['Soil_Type_11', 0.0020951063]
['Soil_Type_3', 0.0019797795]
['Soil_Type_20', 0.0019105831]
['Soil_Type_16', 0.0018759851]
['Soil_Type_39', 0.00172606]
['Soil_Type_22', 

In [11]:
xgb_base= joblib.load('xgb_train_sklearn_0905_drop.model')
train_new_feature = xgb_base.apply(X_drop)
train_x_new,val_x_new,train_y_new,val_y_new = train_test_split(train_new_feature,Y,test_size = 0.3,random_state=0)
def xgb_train_0901_new_feature_select(args):  
    global train_x_new,val_x_new,train_y_new,val_y_new,count,epoch_acc
    params = args
    params['max_depth'] = params.get('max_depth')+10
    params['n_estimators'] = params.get('n_estimators') * 5 + 50
    params['reg_lambda'] = max(1,params.get('reg_lambda'))
    params['subsample'] = params.get('subsample') * 0.1 + 0.7
    params['seed'] = max(1,params.get('seed') )* 100
    if count >0:
        print('\n')
    print(params)
    
    params.setdefault('objective','multi:softmax')
    params.setdefault('num_class',7)
    params.setdefault('silent',1)
    params.setdefault('eval_metric','merror')
#     params.setdefault('seed',900)
    params.setdefault('max_delta_step',30)
    

    try:
        clf = XGBClassifier(**params)
        model_bst =  clf.fit(train_x_new, train_y_new)
#        model_bst = xgb.train(params, d_train_0820, 800)
#        ypred = model_bst.predict(val_x_0821)
#        ypred = model_bst.predict(d_test_0820, ntree_limit=model_bst.best_ntree_limit)
    except:
        print('error!')
        score=0
    else:
        ypred = model_bst.predict(val_x_new)
        count = count + 1
        score = metrics.f1_score(val_y_new,ypred,average='weighted')#(val_y.values, y_test_by)
        if epoch_acc < score:
            joblib.dump(model_bst,'xgb_train_0905_apply_feature.model')
            epoch_acc = score
        print("the %s times, the accuary of test dataset(new_feature_select)：" % str(count),score)
        print("Time consuming: ",round(time.time()-start_time,2),"(s)")
    return -score
      

space = {"max_depth":hp.randint("max_depth",21),
         "n_estimators":hp.randint("n_estimators",10),  #[0,1,2,3,4,5] -> [50,]
         "learning_rate":hp.uniform("eta",0.01, 0.15),  
         "subsample":hp.randint("subsample",4),#[0,1,2,3] -> [0.7,0.8,0.9,1.0]
         "reg_lambda":hp.randint("reg_lambda",15), #
         "reg_alpha":hp.randint("reg_alpha",3), 
         "scale_pos_weight":hp.uniform("scale_pos_weight", 0.1, 1),
         "gamma":hp.uniform("gamma", 0.1,0.9),
         "colsample_bytree":hp.uniform("colsample_bytree", 0.1,1),
         "seed":hp.randint("seed",12),
        }
start_time = time.time()
count = 0
epoch_acc = 0
print('=============================begin=============================')
best = fmin(xgb_train_0901_new_feature_select, space ,algo = partial(tpe.suggest,n_startup_jobs=1),max_evals=20)
space_eval(space,best)
print('\n\n'+"Bayesian hyperopt acc :    %f" % - xgb_train_0901_new_feature_select(space_eval(space , best)))
cost_time = time.time()-start_time
print ('\n'+"Bayesian hyperopt on xgboost finished!",'\n',"the total elapsed time :",round(cost_time,2),"(s)")
print('the best params: ','\n',best)
print('=============================over=============================')

=============================begin=============================
{'colsample_bytree': 0.33823761051599627, 'gamma': 0.2300326976122536, 'learning_rate': 0.06388606413227302, 'max_depth': 11, 'n_estimators': 65, 'reg_alpha': 2, 'reg_lambda': 2, 'scale_pos_weight': 0.4804479962680982, 'seed': 700, 'subsample': 0.7}


/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


the 1 times, the accuary of test dataset(new_feature_select)： 0.7394997281304037
Time consuming:  460.52 (s)


{'colsample_bytree': 0.2631782995654688, 'gamma': 0.1577872208489784, 'learning_rate': 0.056747877538176006, 'max_depth': 11, 'n_estimators': 65, 'reg_alpha': 2, 'reg_lambda': 2, 'scale_pos_weight': 0.4648963046358112, 'seed': 700, 'subsample': 0.7}


/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


the 2 times, the accuary of test dataset(new_feature_select)： 0.7344118951537848
Time consuming:  826.52 (s)


{'colsample_bytree': 0.9522429635718139, 'gamma': 0.7789208067079345, 'learning_rate': 0.1375501663962673, 'max_depth': 25, 'n_estimators': 60, 'reg_alpha': 2, 'reg_lambda': 2, 'scale_pos_weight': 0.973119071327909, 'seed': 1100, 'subsample': 0.7}


...

/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


the 19 times, the accuary of test dataset(new_feature_select)： 0.7603823841515702
Time consuming:  26624.46 (s)


{'colsample_bytree': 0.757906939344507, 'gamma': 0.6892100262105991, 'learning_rate': 0.10486339247893624, 'max_depth': 15, 'n_estimators': 85, 'reg_alpha': 1, 'reg_lambda': 3, 'scale_pos_weight': 0.6691145020746394, 'seed': 400, 'subsample': 1.0}


/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


the 20 times, the accuary of test dataset(new_feature_select)： 0.760876157160956
Time consuming:  28211.5 (s)


{'colsample_bytree': 0.8671699566907659, 'gamma': 0.8936295745545387, 'learning_rate': 0.13471706141584866, 'max_depth': 12, 'n_estimators': 85, 'reg_alpha': 1, 'reg_lambda': 3, 'scale_pos_weight': 0.6507512710079562, 'seed': 400, 'subsample': 1.0}
the 21 times, the accuary of test dataset(new_feature_select)： 0.7632204159327571
Time consuming:  29646.8 (s)


Bayesian hyperopt acc :    0.763220

Bayesian hyperopt on xgboost finished! 
 the total elapsed time : 29646.83 (s)
the best params:  
 {'colsample_bytree': 0.8671699566907659, 'eta': 0.13471706141584866, 'gamma': 0.8936295745545387, 'max_depth': 2, 'n_estimators': 7, 'reg_alpha': 1, 'reg_lambda': 3, 'scale_pos_weight': 0.6507512710079562, 'seed': 4, 'subsample': 3}
=============================over=============================


/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
sorted_idx = np.argsort(xgb_base.feature_importances_)[::-1]
for index in sorted_idx:
    print([pd.DataFrame(X).columns[index],xgb_base.feature_importances_[index]])

['Horizontal_Distance_To_Fire_Points', 0.15797679]
['Hillshade_9am', 0.09391929]
['Horizontal_Distance_To_Roadways', 0.09193965]
['Elevation', 0.09145554]
['Aspect', 0.09110975]
['Hillshade_3pm', 0.08948166]
['Hillshade_Noon', 0.08768932]
['Vertical_Distance_To_Hydrology', 0.073912494]
['Horizontal_Distance_To_Hydrology', 0.070489176]
['Slope', 0.0615851]
['Ground_position', 0.04591219]
['Wilderness_Area_1', 0.003918947]
['Wilderness_Area_3', 0.003446368]
['Soil_Type_33', 0.003221605]
['Wilderness_Area_2', 0.0029651443]
['Soil_Type_32', 0.0027663156]
['Soil_Type_10', 0.0024522233]
['Soil_Type_24', 0.0023715391]
['Soil_Type_31', 0.002311026]
['Soil_Type_29', 0.0022274603]
['Soil_Type_23', 0.0019133682]
['Soil_Type_20', 0.0016943682]
['Soil_Type_2', 0.0016280919]
['Soil_Type_4', 0.0016223288]
['Soil_Type_13', 0.0014321445]
['Soil_Type_3', 0.0013860393]
['Soil_Type_11', 0.0013312893]
['Soil_Type_39', 0.0012938288]
['Soil_Type_30', 0.0012246709]
['Soil_Type_22', 0.0012217894]
['Soil_Type_3

In [ ]:
new_train_data = train_new_feature[train_new_feature.columns[sorted_idx[:160]]]
new_test_data = test_new_feature[test_new_feature.columns[sorted_idx[:160]]]

In [14]:
test_new_feature = xgb_base.apply(test_data.drop(['Ground_position','Wilderness_Area_4','Soil_Type_9','Soil_Type_28'],axis=1))
new_test_y = xgb_new_model.predict(test_new_feature)
np.savetxt('xgb_sklearn_0905_new_feature_drop.csv',new_test_y ,delimiter=',',comments='',fmt='%d')

/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [38]:
b=np.load('0903_result_array.npy')
result= b+ xgb_new_model.predict_proba(test_new_feature)
#np.savetxt('rf_train_new_feature_0904_mix.csv',result+1,delimiter=',',comments='',fmt='%d')

In [40]:
result = np.argmax(result,axis=1)
np.savetxt('rf_train_new_feature_0904_mix.csv',result+1,delimiter=',',comments='',fmt='%d')

In [3]:
xgb_base= joblib.load('xgb_sklearn_0821.model')
datas=datas.sample(frac=1, replace=True)
Y = datas['Cover_Type']
X = datas.drop(['Cover_Type'],axis=1)
train_new_feature = pd.DataFrame(xgb_base.apply(np.array(X)))
test_new_feature = pd.DataFrame(xgb_base.apply(np.array(test_data)))

C:\Program Files\Anaconda3\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.19.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
train_x,val_x,train_y,val_y = train_test_split(train_new_feature,Y,test_size = 0.3,random_state=1)

In [9]:
clf = RandomForestClassifier(n_estimators=150)
clf.fit(train_x,train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [10]:
y_pred = clf.predict(val_x)
metrics.f1_score(val_y,y_pred,average='weighted')

0.86593728466387365

In [11]:
new_test_y_rf = clf.predict(test_new_feature)
np.savetxt('xgb_train_new_feature_rf_0826.csv',new_test_y_rf  ,delimiter=',',comments='',fmt='%d')

In [6]:
y_base = xgb_base.predict(np.array(X))
print('the base model result is :'+str(metrics.f1_score(Y,y_base,average='weighted')))

the base model result is :0.814305556892


In [7]:
xgb_bst_0823 = joblib.load('xgb_sklearn_0821_new_feature.model')
sorted_idx = np.argsort(xgb_bst_0823.feature_importances_)[::-1]
new_train_data = train_new_feature[train_new_feature.columns[sorted_idx[:160]]]
new_test_data = test_new_feature[test_new_feature.columns[sorted_idx[:160]]]

C:\Program Files\Anaconda3\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.19.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
y_new_feature = xgb_bst_0823.predict(np.array(train_new_feature))
print('the new_feature model result is :'+str(metrics.f1_score(Y,y_new_feature,average='weighted')))

the base model result is :0.900356719508


In [5]:
xgb_adasyn_0825 = joblib.load('xgb_train_0825_new_feature_adasyn.model')
new_test_y_0825 = xgb_adasyn_0825.predict(test_new_feature)
np.savetxt('xgb_train_new_feature_adasyn_0825.csv',new_test_y_0825 ,delimiter=',',comments='',fmt='%d')

C:\Program Files\Anaconda3\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.19.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [11]:
new_test_y = xgb_bst_0823.predict(np.array(test_new_feature))
np.savetxt('xgb_sklearn_0821_new_feature_23.csv',new_test_y ,delimiter=',',comments='',fmt='%d')

In [9]:
xgb_new_feature_select_0823 = joblib.load('xgb_train_0823_new_feature_select.model')
y_new_feature_select = xgb_new_feature_select_0823.predict(new_train_data)
print('the new_feature model select  result is :'+str(metrics.f1_score(Y,y_new_feature_select,average='weighted')))

C:\Program Files\Anaconda3\lib\site-packages\sklearn\base.py:312: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.19.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


the new_feature model select  result is :0.843266569253


In [7]:
xgb_new_feature_select_0823 = joblib.load('xgb_train_0823_new_feature_select.model')
new_test_y = xgb_new_feature_select_0823.predict(new_test_data)

/root/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
np.savetxt('xgb_new_feature_select_.csv',xgb_new_feature_select_0823,delimiter=',',comments='',fmt='%d')

ValueError: Expected 1D or 2D array, got 0D array instead